In [ ]:
%load_ext autoreload
%autoreload 2
%run notebook_setup.py

In [ ]:
import sys
import os
from backtest import *

In [ ]:
config = load_config('configs/template.json')
{k: config[k] for k in ['backtest', 'bot', 'live']}

In [ ]:
#config['backtest']['symbols'] = config['backtest']['symbols'][::10]
config['backtest']['start_date'] = '2024-03-01'


In [ ]:
symbols = config['backtest']['symbols']
symbols, hlcvs, mss, results_path = await prepare_hlcvs_mss(config)

In [ ]:
#config['bot']['long']['n_positions'] = 3
#config['bot']['long']['filter_rolling_window'] = 1440
#config['bot']['long']['filter_relative_volume_clip_pct'] = 0.5

In [ ]:
fills, equities, analysis = run_backtest(hlcvs, mss, config)

In [ ]:
sts = utc_ms()
fdf = process_forager_fills(fills)
print(f'elapsed {utc_ms() - sts}')
sts = utc_ms()
equities = pd.Series(equities)
analysis_py, balance_and_equity = analyze_fills_forager(config['backtest']['symbols'], hlcvs, fdf, equities)
for k in analysis_py:
    if k not in analysis:
        analysis[k] = analysis_py[k]
print(f'elapsed {utc_ms() - sts}')
pprint.pprint(analysis)
balance_and_equity.plot()

In [ ]:
syms_sorted_by_volume = fdf.groupby('symbol').fee_paid.sum().sort_values().index.to_list()
for i, symbol in enumerate(syms_sorted_by_volume[:5]):
    print(f"Plotting fills for {symbol}")
    hlcs_df = pd.DataFrame(hlcs[:, symbols.index(symbol), :], columns=["high", "low", "close"])
    fdfc = fdf[fdf.symbol == symbol]
    plt.clf()
    plot_fills_forager(fdfc, hlcs_df)
    plt.title(f"Fills {symbol}")
    plt.xlabel = "time"
    plt.ylabel = "price"
    plt.show()
